##### Mount Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##### Import Libraries

In [ ]:
from tensorflow.keras import layers, optimizers, losses, metrics, activations, regularizers, callbacks
from keras.models import Model
import numpy as np

##### Import Data

In [ ]:
path = 'data path'
x_train = np.load(path + 'x_train.npy')
y_train = np.load(path + 'y_train.npy')
x_test  = np.load(path + 'x_test.npy')
y_test  = np.load(path + 'y_test.npy')

x_train = x_train.transpose(0, 2, 1)            # transpose working correctly
x_test  = x_test.transpose(0, 2, 1)

x_train = x_train.reshape(19634, 12, 1000, 1)   # Add another channel
x_test  = x_test.reshape(2203, 12, 1000, 1)

print("x_train :", x_train.shape)
print("y_train :", y_train.shape)
print("x_test  :", x_test.shape)
print("y_test  :", y_test.shape)
print('Data loaded')

x_train : (19634, 12, 1000, 1)
y_train : (19634, 5)
x_test  : (2203, 12, 1000, 1)
y_test  : (2203, 5)
Data loaded


##### Model

In [ ]:
input = layers.Input(shape=(12, 1000, 1))

conv1 = layers.Conv2D(filters=32, kernel_size=(1, 5))(input)
norm1 = layers.BatchNormalization()(conv1)
relu1 = layers.ReLU()(norm1)
pool1 = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(relu1)

convC1 = layers.Conv2D(filters=64, kernel_size=(1, 7))(pool1)

conv2 = layers.Conv2D(filters=32, kernel_size=(1, 5))(pool1)
norm2 = layers.BatchNormalization()(conv2)
relu2 = layers.ReLU()(norm2)
pool2 = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(relu2)

convC2 = layers.Conv2D(filters=64, kernel_size=(1, 6))(convC1)

conv3 = layers.Conv2D(filters=64, kernel_size=(1, 5))(pool2)
norm3 = layers.BatchNormalization()(conv3)
conv3 = layers.Add()([convC2, norm3])           # skip Connection
relu3 = layers.ReLU()(norm3)
pool3 = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(relu3)

convE1 = layers.Conv2D(filters=32, kernel_size=(1, 4))(pool3)

conv4 = layers.Conv2D(filters=64, kernel_size=(1, 3))(pool3)
norm4 = layers.BatchNormalization()(conv4)
relu4 = layers.ReLU()(norm4)
pool4 = layers.MaxPooling2D(pool_size=(1, 4), strides=1)(relu4)

convE2 = layers.Conv2D(filters=64, kernel_size=(1, 5))(convE1)

conv5 = layers.Conv2D(filters=64, kernel_size=(1, 3))(pool4)
norm5 = layers.BatchNormalization()(conv5)
conv5 = layers.Add()([convE2, norm5])         # skip Connection
relu5 = layers.ReLU()(norm5)
pool5 = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(relu5)

print('Added 5 layers for temporal analysis')

conv6 = layers.Conv2D(filters=64, kernel_size=(12, 1))(pool5)
norm6 = layers.BatchNormalization()(conv6)
relu6 = layers.ReLU()(norm6)
pool6 = layers.MaxPooling2D(pool_size=(1, 2), strides=1)(relu6)
print('Added 1 layer for spatial Analysis')

flat7 = layers.Flatten()(pool6)

dense8 = layers.Dense(units=128, kernel_regularizer=regularizers.L2(0.005))(flat7)
norm8  = layers.BatchNormalization()(dense8)
relu8  = layers.ReLU()(norm8)
drop8  = layers.Dropout(rate=0.2)(relu8)

dense9 = layers.Dense(units=64, kernel_regularizer=regularizers.L2(0.009))(drop8)
norm9 = layers.BatchNormalization()(dense9)
relu9 = layers.ReLU()(norm9)
drop9 = layers.Dropout(rate=0.25)(relu9)
print('Added 2 fully connected layers')

output = layers.Dense(5, activation='sigmoid')(drop9)
model = Model(inputs=input, outputs=output)
print(model.summary())

##### Train Model

In [ ]:
# Source: https://keras.io/api/callbacks/
# Source: https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

early    = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
reducelr = callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3)
callback = [early, reducelr]

model.compile(optimizer = optimizers.Adam(learning_rate=0.0005), 
              loss = losses.BinaryCrossentropy(),
              metrics = [metrics.BinaryAccuracy(), metrics.AUC(curve='ROC', multi_label=True)])

history = model.fit(x_train, y_train, validation_split=0.12, epochs=20, batch_size=64, callbacks=callback)

##### Save Model

In [ ]:
save_path = 'save path'
model.save(save_path + "ST-CNN-5.h5")

##### Evaluate Model

In [ ]:
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score, accuracy_score, auc 


def sklearn_metrics(y_true, y_pred):
    y_bin = np.copy(y_pred)
    y_bin[y_bin >= 0.5] = 1
    y_bin[y_bin < 0.5]  = 0

    # Compute area under precision-Recall curve
    auc_sum = 0
    for i in range(5):
      precision, recall, thresholds = precision_recall_curve(y_true[:, i], y_pred[:,i])
      auc_sum += auc(recall, precision) 

    print("Accuracy        : {:.2f}".format(accuracy_score(y_true.flatten(), y_bin.flatten())* 100))
    print("Macro AUC score : {:.2f}".format(roc_auc_score(y_true, y_pred, average='macro') * 100))
    print('AUPRC           : {:.2f}'.format((auc_sum / 5) * 100))
    print("Micro F1 score  : {:.2f}".format(f1_score(y_true, y_bin, average='micro') * 100))

In [ ]:
y_pred_train = model.predict(x_train)
y_pred_test  = model.predict(x_test)

In [ ]:
print("Train")
sklearn_metrics(y_train, y_pred_train)
print("\nTest")
sklearn_metrics(y_test, y_pred_test)